In [ ]:
import pandas as pd
import plotly.express as px
import sys
sys.path.append('../')

from utils.underlying import ETF
from utils.opions.european import (
    EuropeanCall,
    EuropeanPut,
    EuropeanOption
)

df = pd.read_csv("../data/raw/qqq.csv")
underlying_price = pd.read_csv("../data/raw/qqq_underlying.csv")
underlying_price = pd.Series(
    underlying_price[["0"]].values.flatten(),
    index=underlying_price[["Unnamed: 0"]].values.flatten()
)

df.QUOTE_DATE = pd.to_datetime(df.QUOTE_DATE)
df = df[df.QUOTE_DATE>"2022-01-01"].reset_index(drop=True)
print(df.shape)
df.head()

In [ ]:
df.QUOTE_DATE.max()

## Underlying Price

### Initalize ETF Class

In [ ]:
qqq = ETF(
    prices=underlying_price,
    sym="QQQ"
)
underlying_data = qqq.analyze_index_prices()
underlying_data.head()

### Underlying Price Time Series 

In [ ]:
qqq.plot_price()

### Returns 

In [ ]:
qqq.plot_returns()

### 30 Day Annualized Realized Volatility

In [ ]:
qqq.plot_realized_volatility()

## Options Data

### Call Data

In [ ]:
df_calls = df[["QUOTE_DATE","UNDERLYING_LAST","EXPIRE_DATE","DTE","STRIKE","C_IV","C_LAST","C_BID","C_ASK","C_DELTA"]]
df_calls = df_calls[df_calls.DTE>0]
df_calls.head()

#### Bid Ask Spread & Last Price 

In [ ]:
df_calls["In_ba_spread"] =  (df_calls.C_BID < df_calls.C_LAST) & (df_calls.C_LAST < df_calls.C_ASK)
tmp = df_calls[["C_DELTA","In_ba_spread"]]
tmp.C_DELTA = tmp.C_DELTA.round(2)
tmp = tmp.groupby("C_DELTA").mean().reset_index()
fig = px.scatter(
    tmp,
    x="C_DELTA",
    y="In_ba_spread",
    template="simple_white"
)
fig.show()

#### Distribution of DTE

In [ ]:
print(df_calls.DTE.describe())
fig = px.histogram(
    df_calls,
    x="DTE",
    template="simple_white"
)
fig.show()

#### Distribution of IV

In [ ]:
print(df_calls.C_IV.describe())
fig = px.histogram(
    df_calls,
    x="C_IV",
    template="simple_white"
)
fig.show()

#### IV & DTE

In [ ]:
tmp = df[["DTE","C_IV"]].groupby("DTE").agg({"C_IV": ["mean","std"]}).reset_index()
tmp.columns = ["DTE","IV_Mean","sigma"]
tmp["PlusOneSigma"] = tmp.IV_Mean + tmp.sigma
tmp["MinusOneSigma"] = tmp.IV_Mean - tmp.sigma
fig = px.line(
    tmp,
    x="DTE",
    y=["IV_Mean","PlusOneSigma","MinusOneSigma"],
    template="simple_white"
)
fig.show()

#### Missing IV and DTE

In [ ]:
tmp = df[["DTE","C_IV"]]
tmp.C_IV = tmp.C_IV.isna()
tmp = tmp.groupby("DTE").mean().reset_index()
fig = px.scatter(
    tmp,
    x="DTE",
    y="C_IV",
    template="simple_white"
)
fig.show()

### Put Data

In [ ]:
df_puts = df[["QUOTE_DATE","UNDERLYING_LAST","EXPIRE_DATE","DTE","STRIKE","P_IV","P_LAST","P_BID","P_ASK","P_DELTA"]]
df_puts = df_puts[df_puts.DTE>0]
df_puts.head()

#### Bid Ask Spread & Last Price 

In [ ]:
df_puts["In_ba_spread"] =  (df_puts.P_BID < df_puts.P_LAST) & (df_puts.P_LAST < df_puts.P_ASK)
tmp = df_puts[["P_DELTA","In_ba_spread"]]
tmp.P_DELTA = tmp.P_DELTA.round(2)
tmp = tmp.groupby("P_DELTA").mean().reset_index()
fig = px.scatter(
    tmp,
    x="P_DELTA",
    y="In_ba_spread",
    template="simple_white"
)
fig.show()

#### Distribution of DTE

In [ ]:
print(df_puts.DTE.describe())
fig = px.histogram(
    df_puts,
    x="DTE",
    template="simple_white"
)
fig.show()

#### Distribution of IV

In [ ]:
print(df_puts.P_IV.describe())
fig = px.histogram(
    df_puts,
    x="P_IV",
    template="simple_white"
)
fig.show()

#### IV & DTE

In [ ]:
tmp = df_puts[["DTE","P_IV"]].groupby("DTE").agg({"P_IV": ["mean","std"]}).reset_index()
tmp.columns = ["DTE","IV_Mean","sigma"]
tmp["PlusOneSigma"] = tmp.IV_Mean + tmp.sigma
tmp["MinusOneSigma"] = tmp.IV_Mean - tmp.sigma
fig = px.line(
    tmp,
    x="DTE",
    y=["IV_Mean","PlusOneSigma","MinusOneSigma"],
    template="simple_white"
)
fig.show()

#### Missing IV and DTE

In [ ]:
tmp = df[["DTE","P_IV"]]
tmp.P_IV = tmp.P_IV.isna()
tmp = tmp.groupby("DTE").mean().reset_index()
fig = px.scatter(
    tmp,
    x="DTE",
    y="P_IV",
    template="simple_white"
)
fig.show()